Team Name: EnviroMinds
Team Members: Sanjana Sudarsan, Swetha Sriram, Lohithaa K M
Leaderboard Rank: 53

In [ ]:
pip install efficientnet_pytorch

install

In [ ]:
# inference.py
import os, cv2, torch, pandas as pd, numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

# Configs
DATA_DIR = "/kaggle/input/soil-classification-1/soil_classification-2025"
TEST_IMG = os.path.join(DATA_DIR, "test")
TEST_IDS = os.path.join(DATA_DIR, "test_ids.csv")
OUTPUT_DIR = "/kaggle/working"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE, NUM_CLASSES, N_FOLDS = 32, 4, 5
idx2label = {0: 'Alluvial soil', 1: 'Black Soil', 2: 'Clay soil', 3: 'Red soil'}

def load_clahe_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (300, 300))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(2.0, (8,8)).apply(l)
    return Image.fromarray(cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2RGB))

transform_val = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class TestDataset(Dataset):
    def _init_(self, image_ids, img_dir, transform):
        self.image_ids, self.img_dir, self.transform = image_ids, img_dir, transform
    def _len_(self): return len(self.image_ids)
    def _getitem_(self, idx):
        img_id = self.image_ids[idx]
        img = load_clahe_image(os.path.join(self.img_dir, img_id))
        return self.transform(img), img_id

test_ids = pd.read_csv(TEST_IDS)
test_list = test_ids['image_id'].tolist()
test_loader = DataLoader(TestDataset(test_list, TEST_IMG, transform_val), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

ensemble_probs = {img_id: [] for img_id in test_list}
for fold in range(N_FOLDS):
    model = EfficientNet.from_name('efficientnet-b3')
    model._fc = torch.nn.Linear(model._fc.in_features, NUM_CLASSES)
    model.load_state_dict(torch.load(f"{OUTPUT_DIR}/model_fold{fold}.pth"))
    model = model.to(DEVICE).eval()

    with torch.no_grad():
        for imgs, img_ids in test_loader:
            imgs = imgs.to(DEVICE)
            probs = model(imgs).cpu().softmax(1).numpy()
            for i in range(len(img_ids)):
                ensemble_probs[img_ids[i]].append(probs[i])

# Write submission
out = []
for img_id in test_list:
    avg = np.mean(ensemble_probs[img_id], axis=0)
    out.append({'image_id': img_id, 'soil_type': idx2label[int(avg.argmax())]})

pd.DataFrame(out).to_csv(f"{OUTPUT_DIR}/submission.csv", index=False)
print("submission.csv written")

inference